In [1]:
from rpy2.robjects import DataFrame, FloatVector, IntVector
from rpy2.robjects.packages import importr

r_icc = importr("ICC")

import numpy as np
import pandas as pd
import random
RS = 2020
random.seed(RS)
np.random.seed(RS)

from KMeans import KMeansBalanced

def pashuffle(string, perc=10):
    random.seed(perc)
    data = list(string)
    for index, letter in enumerate(data):
        if random.randrange(0, 100) < perc:
            new_index = random.randrange(0, len(data))
            data[index], data[new_index] = data[new_index], data[index]
    return data

cffi mode is CFFI_MODE.ANY
Unable to determine R home: [WinError 2] The system cannot find the file specified
R home found: C:\Program Files\R\R-4.2.0
R library path: 
LD_LIBRARY_PATH: 
Default options to initialize R: rpy2, --quiet, --no-save


In [2]:
df1 = pd.read_csv("Dataset.csv")
old_list = []
for shuffle_amount in range(0,101,5):
    groups = pashuffle(df1["level"],shuffle_amount)
    values = df1["y"]
    df = DataFrame({"groups": IntVector(groups),
                    "values": FloatVector(values)})
    icc_res = r_icc.ICCbare("groups", "values", data=df)
    icc_val = icc_res[0]
    old_list.append(icc_val)
print(f"ICC as noise increases: {old_list}")
X = np.array(df1.iloc[:,:2])
cl2 = KMeansBalanced(k=len(df1)//15)
y_pred, centroids = cl2.fit(X)
new_list =[]
for shuffle_amount in range(0,101,5):
    df_temp = df1.copy()
    df_temp["level"] = pashuffle(df_temp["level"],shuffle_amount)
    icc_val_list=[]
    for i in range(20):
        df2 = df_temp[np.array(y_pred)==i]
        groups = df2["level"]
        values = df2["y"]
        df = DataFrame({"groups": IntVector(groups),
                        "values": FloatVector(values)})
        icc_res = r_icc.ICCbare("groups", "values", data=df)
        icc_val = icc_res[0]
        icc_val_list.append(icc_val)
    new_list.append(np.average(icc_val_list))
print(f"MLICC as noise increase: {new_list}")

ICC as noise increases: [0.047033163895090706, 0.05052984845996382, 0.04166031514586037, 0.04903900498379722, 0.012902912516671578, 0.005651439037378947, 0.024877559808263092, 0.00495096605449916, 0.0012720483555524419, -0.007523581401828761, -0.0034557122271565966, -0.004274843416706284, 0.01119381740221536, 0.008133327336461259, -0.003947137380977288, -0.007887359225973908, -0.007309327048256554, -0.0025378186133968948, -0.0048773089423377325, 0.01973452677188814, -0.007157701244577041]
MLICC as noise increase: [0.7355786167977469, 0.5493173364057575, 0.4866467045427972, 0.39465943479490784, 0.2930496116312601, 0.2507454856036335, 0.22733763727863598, 0.17244890783409692, 0.15197902581326234, 0.16271781531661783, 0.03294769400888484, -0.0005388602044879289, 0.004075234285701945, 0.006534848605489455, 0.0015941797790157148, -0.001508757578448805, -0.013598386455152262, 0.0030265555564995814, -0.023091513923471533, -0.05844569057833602, -0.07818999155564763]
